In [7]:

import random
import numpy as np
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import gc
from matplotlib import pyplot as plt
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import xgboost as xgb
import pickle

def init(load_path):
  with open(load_path+'uid_user.pkl', 'rb') as f:
      uid_user = pickle.load(f)
  with open(load_path+'mid_movie.pkl', 'rb') as f:
      mid_movie = pickle.load(f)
  with open(load_path+'movie_mid.pkl', 'rb') as f:
      movie_mid = pickle.load(f)
  with open(load_path+'user_uid.pkl', 'rb') as f:
      user_uid = pickle.load(f)

  # global xgb_model
  xgb_model = xgb.Booster()
  xgb_model.load_model(load_path+'xgb_model.json')

  df_mm = pd.read_csv(load_path+"movie_all_features.csv", index_col=0)
  df_uu = pd.read_csv(load_path+"user_all_features.csv", index_col=0)
  df_mm['mid'] = df_mm['movie_id'].apply(lambda x:movie_mid[x])
  df_mm      = df_mm.drop(columns=['movie_id'])
  df_mm = pd.get_dummies(df_mm).reset_index(drop=True)
  xgb_features = pd.read_csv(load_path+"xgb_features.csv", index_col=0)
  return uid_user, mid_movie, movie_mid, user_uid, xgb_model, df_mm, df_uu, xgb_features


def date_encoding(data):
  data['day_of_week'] = pd.to_datetime(data['time']).dt.dayofweek.astype(int)
  data['is_weekend'] = (data['day_of_week'] >= 5).astype(int)
  data['hour'] = pd.to_datetime(data['time']).dt.hour.astype(int)
  data['hour_sin'] = np.sin(data['hour'] * (2. * np.pi / 24))
  data['hour_cos'] = np.cos(data['hour'] * (2. * np.pi / 24))
  data['day_of_week_sin'] = np.sin(data['day_of_week'] * (2. * np.pi / 7))
  data['day_of_week_cos'] = np.cos(data['day_of_week'] * (2. * np.pi / 7))
  data['release_date'] = pd.to_datetime(data['release_date']).dt.year

  return data.drop(columns=['time'])

def inference_set_up(user_id, df_user, df_movie, \
                     user_mapping,  movie_mapping, feature_name, time='2023-12-27 16:23:09'):
  df_merged = df_movie
  user_feature = df_user[df_user['user_id']==user_id].reset_index(drop=True).iloc[0:1]
  if len(user_feature)==0:
    u_col = list(df_user.columns)[1:]
    u_fea = [20.]+[0.]*(len(u_col)-1)
  for column in user_feature.columns:
    df_merged[column] = user_feature[column].values[0]   #append this user's feature to all movies
  df_merged['time'] =[time]* len(df_merged) ##append time feature to all movies
  df_merged = date_encoding(df_merged)
  X_inference = df_merged.loc[:, list(feature_name.columns)]

  return X_inference

def inference(user_id, X_inference, movie_mapping , model, number_rec=20):
  predictions  = model.predict(xgb.DMatrix(X_inference))
  sorted_indices = np.argsort(-predictions)[:number_rec]

  return  predictions[sorted_indices], sorted_indices, [movie_mapping[i] for i in sorted_indices]



In [8]:
import time
start = time.time()
uid_user, mid_movie, movie_mid, user_uid, xgb_model, df_mm, df_uu, xgb_features = init('/content/drive/MyDrive/movie_rec/')

start1 = time.time()
X_inference=inference_set_up(1, df_uu, df_mm, user_uid,movie_mid, xgb_features, '2023-12-27 16:23:09')
rate, ind, rec = inference(1, X_inference, movie_mapping = mid_movie, model=xgb_model, number_rec = 20)
print(rec)
print('Inference Loading file time:', time.time()-start, "Seconds")
print('Inference Prediction time:', time.time()-start1, "Seconds")

['wild+america+1997', 'grandmother+2009', 'mud+2013', 'the+song+of+sparrows+2008', 'penelope+2006', 'pornorama+2007', 'bloody+territories+1969', 'sun+on+the+horizon+1996', 'billy+elliot+2000', 'war+of+the+worlds+2+the+next+wave+2008', 'amish+grace+2010', 'into+the+night+1985', 'wild+target+2010', 'the+unknown+woman+2011', 'carrie+1952', '8+1963', 'the+celebration+1998', 'love+hina+christmas+special+silent+eve+2000', 'the+set-up+1949', 'knightriders+1981']
Inference Loading file time: 1.5694942474365234 Seconds
Inference Prediction time: 0.22533273696899414 Seconds
